In [ ]:
# --- 1. Импорты и общие настройки ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet  # pip install prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")
# plt.style.use("seaborn-whitegrid")



In [2]:
# --- 2. Загрузка данных ---
# df должен содержать столбцы: "Регион", "Период" (YYYY-MM), и целевой столбец с показателем.
df = pd.read_excel("Датасет по овцы и козы.xlsx")
df["Период"] = pd.to_datetime(df["Период"], format="%Y-%m")
df.sample(10)



,Регион,Период,Овцы и козы
392,АТЫРАУСКАЯ ОБЛАСТЬ,2017-09-01,1266.37
1285,КОСТАНАЙСКАЯ ОБЛАСТЬ,2015-07-01,235.75
885,ГШЫМКЕНТ,2022-03-01,70.18
1542,МАНГИСТАУСКАЯ ОБЛАСТЬ,2016-12-01,386.86
429,АТЫРАУСКАЯ ОБЛАСТЬ,2020-10-01,942.03
786,ГАСТАНА,2020-07-01,2.50
672,ГАЛМАТЫ,2021-01-01,0.20
910,ГШЫМКЕНТ,2024-04-01,49.00
143,АКТЮБИНСКАЯ ОБЛАСТЬ,2016-12-01,4322.25
572,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2022-09-01,1649.88


In [5]:
regions = df['Регион'].unique()
target   = "Овцы и козы"
horizon  = 3
epsilon = 1e-6

In [6]:
first_test = pd.to_datetime("2024-01-01")
last_possible = df["Период"].max() - pd.DateOffset(months=horizon-1)
test_starts = pd.date_range(first_test, last_possible, freq="MS")

## Holw-Winter's (log)

In [8]:
results_hw = []
for region in regions:
    ts = (df[df["Регион"] == region]
          .set_index("Период")[target]
          .dropna()
          .sort_index())
    if len(ts) < 24:
        print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        # формируем train / test
        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

        # пропускаем, если недостаточно данных или неполный test
        if len(train) < 24 or len(test) < horizon:
            continue

        # обучаем модель
        train_log = np.log1p(train)

        hw_log = ExponentialSmoothing(
            train_log,
            seasonal="add",
            seasonal_periods=12
        ).fit(optimized=True)

        # прогноз и метрики
        fc_log = hw_log.forecast(horizon)
        fc = np.expm1(fc_log) 
        # fc   = model.forecast(horizon)
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test - fc) / test).mean()) * 100

        results_hw.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "Forecast":    [x.round(2) for x in list(fc.values)],
            "Actual":      [y.round(2) for y in list(test.values)],
            "RMSE":        rmse,
            "MAE":         mae,
            "MAPE_%":      mape
        })

# 4) Усреднение по всем скользящим окнам для каждого региона
res_hw = pd.DataFrame(results_hw)
print("Результаты прогнозов HW на 3 месяца:")

display(res_hw)

final_hw = (
    res_hw
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Holt–Winter's по регионам (rolling-3):")
display(final_hw)

Результаты прогнозов HW на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[539.98, 556.64, 585.43]","[544.33, 634.45, 733.22]",96.464957,76.649951,11.073125
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[557.52, 586.08, 653.24]","[634.45, 733.22, 686.11]",97.720776,85.645497,12.327823
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[596.32, 658.54, 769.98]","[733.22, 686.11, 756.58]",80.996883,59.291851,8.153749
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[666.09, 782.4, 777.1]","[686.11, 756.58, 825.36]",33.644766,31.363968,4.058914
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[784.38, 779.21, 505.68]","[756.58, 825.36, 499.73]",31.295096,26.633353,3.485475
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[4123.05, 3623.37, 3899.19]","[4988.31, 2350.88, 3378.76]",937.859841,886.058251,28.958908
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[3973.77, 4269.8, 5128.5]","[2350.88, 3378.76, 4625.15]",1107.712622,1005.760518,35.429327
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[3811.44, 4730.34, 4991.98]","[3378.76, 4625.15, 4443.92]",407.696748,361.978353,9.137700
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[4558.31, 4812.73, 4869.31]","[4625.15, 4443.92, 5402.06]",376.082405,322.801623,6.535474


Средние метрики Holt–Winter's по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,57.42,49.23,7.00
1,АКТЮБИНСКАЯ ОБЛАСТЬ,112.58,94.29,5.63
2,АЛМАТИНСКАЯ ОБЛАСТЬ,411.20,338.01,14.43
3,АТЫРАУСКАЯ ОБЛАСТЬ,66.36,55.91,6.38
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,136.14,115.87,9.09
5,ГАЛМАТЫ,0.41,0.37,NaN
6,ГАСТАНА,0.20,0.17,16.22
7,ГШЫМКЕНТ,35.15,25.06,24.58
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,177.04,166.39,6.20
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,26.80,23.68,2.01


## SARIMA

In [9]:
results_sarima = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12

        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            D=1 if use_seasonal else 0,      # фиксируем порядок сезонной разности
            seasonal_test=None,               # пропустить nsdiffs
            boxcox=True,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
      
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         [round(y,2) for y in actual]
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

Результаты прогнозов SARIMA на 3 месяца:


,Регион,Test start,Test end,order,seasonal_order,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"(1, 0, 0)","(1, 1, 0, 12)",92.15,72.71,10.49,"[539.61, 564.39, 589.88]","[544.33, 634.45, 733.22]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"(1, 0, 0)","(1, 1, 0, 12)",93.28,81.89,11.76,"[566.56, 590.97, 650.59]","[634.45, 733.22, 686.11]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"(1, 0, 0)","(1, 1, 0, 12)",68.53,50.41,6.94,"[617.65, 661.28, 767.43]","[733.22, 686.11, 756.58]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"(1, 0, 0)","(1, 1, 0, 12)",38.55,36.46,4.72,"[707.82, 791.88, 773.0]","[686.11, 756.58, 825.36]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"(1, 0, 0)","(1, 1, 0, 12)",35.87,29.37,3.82,"[781.93, 769.0, 493.32]","[756.58, 825.36, 499.73]"
...,...,...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"(2, 1, 0)","(1, 1, 1, 12)",684.19,627.94,18.90,"[4093.34, 3084.55, 3633.92]","[4988.31, 2350.88, 3378.76]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"(2, 1, 2)","(1, 1, 1, 12)",790.62,740.01,25.09,"[3474.3, 3850.0, 5250.51]","[2350.88, 3378.76, 4625.15]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"(0, 1, 1)","(0, 1, 0, 12)",178.25,173.95,4.29,"[3565.05, 4410.95, 4565.29]","[3378.76, 4625.15, 4443.92]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"(0, 1, 1)","(0, 1, 0, 12)",524.67,394.11,7.64,"[4305.16, 4455.8, 4551.59]","[4625.15, 4443.92, 5402.06]"


Средние метрики SARIMA по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,55.60,48.21,6.860000e+00
1,АКТЮБИНСКАЯ ОБЛАСТЬ,158.30,139.45,8.290000e+00
2,АЛМАТИНСКАЯ ОБЛАСТЬ,385.19,301.24,1.386000e+01
3,АТЫРАУСКАЯ ОБЛАСТЬ,80.87,67.18,7.210000e+00
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,181.82,161.02,1.330000e+01
5,ГАЛМАТЫ,0.40,0.29,1.938946e+21
6,ГАСТАНА,0.14,0.11,9.390000e+00
7,ГШЫМКЕНТ,39.13,27.03,2.468000e+01
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,172.88,152.05,5.760000e+00
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,64.85,59.28,3.970000e+00


## Facebook Prophet

In [15]:
results_prophet = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. данных, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # Подготовка данных для Prophet
#         df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
# # подготовка для одного региона
        df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
        df_prophet["y"] = np.log(df_prophet["y"] + epsilon)

        m = Prophet()
        m.fit(df_prophet)

        future = m.make_future_dataframe(periods=horizon, freq="MS")
        forecast = m.predict(future)

        # берем только прогнозные точки
        yhat_log = forecast["yhat"].values[-horizon:]
        fc = np.exp(yhat_log) - epsilon

        # m = Prophet()
        # m.fit(df_prophet)

        # # Создаем DataFrame будущих дат и делаем прогноз
        # # future = m.make_future_dataframe(periods=horizon, freq="MS")
        # # forecast = m.predict(future)

        # # Отбираем только наши горизонты
        # fc = forecast.set_index("ds")["yhat"].loc[test.index].values
        actual = test.values

        # Расчет метрик
        rmse  = np.sqrt(mean_squared_error(actual, fc))
        mae   = mean_absolute_error(actual, fc)
        mape  = (np.abs((actual - fc) / actual).mean()) * 100

        results_prophet.append({
            "Регион":     region,
            "Test start": test_start.strftime("%Y-%m"),
            "Test end":   test_end.strftime("%Y-%m"),
            "RMSE":       round(rmse, 2),
            "MAE":        round(mae, 2),
            "MAPE_%":     round(mape, 2),
            "Forecast":   [round(x, 2) for x in fc],
            "Actual":     [round(x, 2) for x in actual]
        })

# Собираем результаты в DataFrame
res_prophet = pd.DataFrame(results_prophet)
print("Результаты прогнозов Prophet на 3 месяца:")
display(res_prophet)

final_prophet = (
    res_prophet
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Prophet по регионам (rolling-3):")
display(final_prophet)


17:14:35 - cmdstanpy - INFO - Chain [1] start processing
17:14:35 - cmdstanpy - INFO - Chain [1] done processing
17:14:35 - cmdstanpy - INFO - Chain [1] start processing
17:14:35 - cmdstanpy - INFO - Chain [1] done processing
17:14:35 - cmdstanpy - INFO - Chain [1] start processing
17:14:35 - cmdstanpy - INFO - Chain [1] done processing
17:14:35 - cmdstanpy - INFO - Chain [1] start processing
17:14:35 - cmdstanpy - INFO - Chain [1] done processing
17:14:36 - cmdstanpy - INFO - Chain [1] start processing
17:14:36 - cmdstanpy - INFO - Chain [1] done processing
17:14:36 - cmdstanpy - INFO - Chain [1] start processing
17:14:36 - cmdstanpy - INFO - Chain [1] done processing
17:14:36 - cmdstanpy - INFO - Chain [1] start processing
17:14:36 - cmdstanpy - INFO - Chain [1] done processing
17:14:36 - cmdstanpy - INFO - Chain [1] start processing
17:14:37 - cmdstanpy - INFO - Chain [1] done processing
17:14:37 - cmdstanpy - INFO - Chain [1] start processing
17:14:37 - cmdstanpy - INFO - Chain [1]

Результаты прогнозов Prophet на 3 месяца:


,Регион,Test start,Test end,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,84.49,69.47,10.15,"[560.8, 575.4, 600.34]","[544.33, 634.45, 733.22]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,96.61,90.90,13.06,"[573.17, 596.71, 611.22]","[634.45, 733.22, 686.11]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,84.19,71.73,9.96,"[605.2, 619.35, 777.0]","[733.22, 686.11, 756.58]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,45.54,44.58,6.00,"[628.64, 792.25, 784.77]","[686.11, 756.58, 825.36]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,32.38,30.73,4.32,"[797.97, 791.52, 482.77]","[756.58, 825.36, 499.73]"
...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,1052.38,956.62,32.38,"[4166.84, 3899.37, 3878.68]","[4988.31, 2350.88, 3378.76]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,1042.31,833.85,31.43,"[4029.31, 4017.6, 4809.44]","[2350.88, 3378.76, 4625.15]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,233.24,216.01,5.53,"[3689.74, 4387.01, 4542.82]","[3378.76, 4625.15, 4443.92]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,838.53,586.97,11.23,"[4286.23, 4434.18, 3989.81]","[4625.15, 4443.92, 5402.06]"


Средние метрики Prophet по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,64.97,57.11,8.17
1,АКТЮБИНСКАЯ ОБЛАСТЬ,210.52,197.96,11.57
2,АЛМАТИНСКАЯ ОБЛАСТЬ,926.40,705.28,24.06
3,АТЫРАУСКАЯ ОБЛАСТЬ,65.03,54.95,4.96
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,433.32,379.98,31.59
5,ГАЛМАТЫ,0.42,0.29,NaN
6,ГАСТАНА,0.26,0.21,20.43
7,ГШЫМКЕНТ,40.71,27.82,23.32
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,280.72,243.72,8.11
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,125.40,101.16,6.58


In [19]:
# Переименуем колонки с MAPE, чтобы было понятно, к какому методу относятся
hw = final_hw.rename(columns={"MAPE_%": "MAPE_HW"})
sar = final_sarima.rename(columns={"MAPE_%": "MAPE_SARIMA"})
pr  = final_prophet.rename(columns={"MAPE_%": "MAPE_Prophet"})

# Мёрджим по региону
summary = (
    hw[["Регион", "MAPE_HW"]]
    .merge(sar[["Регион", "MAPE_SARIMA"]], on="Регион")
    .merge(pr[["Регион", "MAPE_Prophet"]], on="Регион")
)

# Определяем для каждой строки, какой столбец MAPE минимален
# idxmin вернёт название столбца с минимальным значением
summary["Best_method"] = summary[["MAPE_HW","MAPE_SARIMA","MAPE_Prophet"]] \
                           .idxmin(axis=1) \
                           .str.replace("MAPE_","")  # убираем префикс для красоты

# Если нужно, можно сразу отсортировать
# summary = summary.sort_values("Best_method")

# допустим, у вас уже есть summary
summary = summary.round({
    "MAPE_HW": 2,
    "MAPE_SARIMA": 2,
    "MAPE_Prophet": 2
})

# Готово!
print(summary.to_string(index=False))


                        Регион  MAPE_HW               MAPE_SARIMA  MAPE_Prophet Best_method
           АКМОЛИНСКАЯ ОБЛАСТЬ     7.00                      6.86          8.17      SARIMA
           АКТЮБИНСКАЯ ОБЛАСТЬ     5.63                      8.29         11.57          HW
           АЛМАТИНСКАЯ ОБЛАСТЬ    14.43                     13.86         24.06      SARIMA
            АТЫРАУСКАЯ ОБЛАСТЬ     6.38                      7.21          4.96     Prophet
ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ     9.09                     13.30         31.59          HW
                       ГАЛМАТЫ      NaN 1938945707226866384896.00           NaN      SARIMA
                       ГАСТАНА    16.22                      9.39         20.43      SARIMA
                      ГШЫМКЕНТ    24.58                     24.68         23.32     Prophet
            ЖАМБЫЛСКАЯ ОБЛАСТЬ     6.20                      5.76          8.11      SARIMA
 ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ     2.01                      3.97          6.58 